### Aim is to visualize NLP reports in a straight forwards manner

Topic Modeling, Summarization and Wordcloud are all powerful tools to find depth meaning of the verbatim.
However, it is lack of interpretable and reliable explanation. It leads to the higher managers misunderstaning and always in doubt on the results of the fancy NLP conclusion.

- https://towardsdatascience.com/visualize-hierarchical-data-using-plotly-and-datapane-7e5abe2686e1
- https://blog.revolutionanalytics.com/2018/06/global-migration-animated-with-r.html

In [64]:
import pandas as pd

filename = "Ha+Noi+Pho"
df = pd.read_csv(f'{filename}.csv')
df

,Review Text,Review Rate,Review Time,search,insert_date
0,"Wonderful food. The taste was unique, the vege...",5,4 months ago,Ha+Noi+Pho,2022-09-17
1,Small place so if you come as a big group you ...,3,2 weeks ago,Ha+Noi+Pho,2022-09-17
2,Definitely authentic Vietnamese food. The soup...,4,2 weeks ago,Ha+Noi+Pho,2022-09-17
3,"For me, so far it is the best Vietnamese resta...",5,8 months ago,Ha+Noi+Pho,2022-09-17
4,I really like this little Vietnamese place! I...,4,a year ago,Ha+Noi+Pho,2022-09-17
...,...,...,...,...,...
394,NaN,5,3 months ago,Ha+Noi+Pho,2022-09-17
395,NaN,5,11 months ago,Ha+Noi+Pho,2022-09-17
396,NaN,5,a year ago,Ha+Noi+Pho,2022-09-17
397,NaN,5,3 years ago,Ha+Noi+Pho,2022-09-17


In [65]:
df_groupby = df.groupby(by='Review Rate').count()['Review Text']
df_groupby = df_groupby / sum(df_groupby)

In [66]:
# Find the ten most common words
rate = 5

context = df[df['Review Rate']==rate]['Review Text'].str.lower().tolist()[0]

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import Counter
import re
words = re.findall(r'\w+', context)
words = [word for word in words if not word in stopwords.words()]

Counter(words).most_common(5)

[nltk_data] Downloading package stopwords to /Users/sgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('wonderful', 1), ('food', 1), ('taste', 1), ('unique', 1), ('vegetables', 1)]

In [67]:
temp = pd.DataFrame(Counter(words).most_common(5), columns=['item', 'weight'])
temp['weight'] = temp['weight'] / sum(temp['weight'])
temp['parent'] = rate
temp

,item,weight,parent
0,wonderful,0.2,5
1,food,0.2,5
2,taste,0.2,5
3,unique,0.2,5
4,vegetables,0.2,5


In [68]:
df_viz = df_groupby.reset_index()
df_viz = df_viz.rename(columns={"Review Rate": "item", "Review Text": "weight"})
df_viz['parent'] = filename.replace("+", ' ')
df_viz

,item,weight,parent
0,1,0.160714,Ha Noi Pho
1,2,0.025000,Ha Noi Pho
2,3,0.021429,Ha Noi Pho
3,4,0.107143,Ha Noi Pho
4,5,0.685714,Ha Noi Pho


In [69]:
# df_viz = pd.concat([topic, df_viz])
# df_viz
temp.item + '1'

0     wonderful1
1          food1
2         taste1
3        unique1
4    vegetables1
Name: item, dtype: object

In [70]:
# Find the ten most common words
for i in range(1,6):
    rate = i

    context = df[df['Review Rate']==rate]['Review Text'].str.lower().tolist()[0]

    words = re.findall(r'\w+', context)
    words = [word for word in words if not word in stopwords.words()]

    temp = pd.DataFrame(Counter(words).most_common(5), columns=['item', 'weight'])
    temp.item = temp.item + '_' + str(i)
    temp['weight'] = temp['weight'] * df_groupby[rate] / sum(temp['weight']) 
    temp['parent'] = rate

    df_viz = pd.concat([df_viz, temp])
    
# df_viz = pd.concat([df_viz, topic])
df_viz = df_viz.reset_index(drop=True)
df_viz

,item,weight,parent
0,1,0.160714,Ha Noi Pho
1,2,0.025000,Ha Noi Pho
2,3,0.021429,Ha Noi Pho
3,4,0.107143,Ha Noi Pho
4,5,0.685714,Ha Noi Pho
5,paid_1,0.037088,1
6,restaurant_1,0.037088,1
7,bowl_1,0.037088,1
8,service_1,0.024725,1
9,sauce_1,0.024725,1


In [71]:
import plotly.express as px
data = dict(
    character=df_viz['item'].tolist(),
    parent=df_viz['parent'].tolist(),
    value=df_viz['weight'].tolist())

fig = px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    branchvalues='total',
)
fig.show()